In [2]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [3]:
articles_df = pd.read_csv('archive/games.csv')


In [4]:
articles_df.head(5)

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True


In [5]:
#articles_df = pd.read_csv('archive/games.csv')

#articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
#articles_df.head(10)
#interactions_df = pd.read_csv(r'archive/recommendations.csv')
#interactions_df.head(10)

output_file_path = r'split_games.csv'
split_df = pd.DataFrame(columns=['app_id', 'user_id', 'review_id', 'attribute', 'value', 'date'])
split_df.to_csv(output_file_path, index=False, mode='w')

# 分块读取并处理数据
for chunk in pd.read_csv('archive/recommendations.csv', chunksize=1000):
    rows = []
    for index, row in chunk.iterrows():
        rows.append({'app_id': row['app_id'], 'user_id': row['user_id'], 'review_id': row['review_id'], 'attribute': 'funny', 'value': row['funny'], 'date': row['date']})
        rows.append({'app_id': row['app_id'], 'user_id': row['user_id'], 'review_id': row['review_id'], 'attribute': 'helpful', 'value': row['helpful'], 'date': row['date']})
        rows.append({'app_id': row['app_id'], 'user_id': row['user_id'], 'review_id': row['review_id'], 'attribute': 'is_recommended', 'value': row['is_recommended'], 'date': row['date']})
        rows.append({'app_id': row['app_id'], 'user_id': row['user_id'], 'review_id': row['review_id'], 'attribute': 'hours', 'value': row['hours'], 'date': row['date']})
    
    # 将当前块数据转换为DataFrame并写入文件
    chunk_split_df = pd.DataFrame(rows)
    chunk_split_df.to_csv(output_file_path, index=False, mode='a', header=False)

print("拆分完成，文件已保存至:", output_file_path)

chunk_split_df.head(100)

拆分完成，文件已保存至: split_games.csv


,app_id,user_id,review_id,attribute,value,date
0,412020,7179083,26268000.0,funny,0,2022-01-31
1,412020,7179083,26268000.0,helpful,0,2022-01-31
2,412020,7179083,26268000.0,is_recommended,True,2022-01-31
3,412020,7179083,26268000.0,hours,29.5,2022-01-31
4,476600,6792325,26268001.0,funny,0,2017-11-23
...,...,...,...,...,...,...
95,386180,2738482,26268023.0,hours,107.2,2021-11-29
96,346110,10394705,26268024.0,funny,0,2022-06-12
97,346110,10394705,26268024.0,helpful,0,2022-06-12
98,346110,10394705,26268024.0,is_recommended,True,2022-06-12


In [25]:
event_type_strength = {
   'hours': 0.3,
   'helpful': 0.1, 
   'funny': 0.1, 
   'is_recommended': 1.0,
#    'COMMENT CREATED': 4.0,  
}

def evaluate_review(row):
   #print(row)
   if row['attribute'] == "is_recommended":
      if row['value'] == "True":
         return 1
   if row['attribute'] == "funny":
      # if row["value"] == "0":
      #    return 0.1
      # else:
      #    return 0.3
      return 0.3*int(row['value'])
   if row['attribute']=="helpful":
      return 0.3*int(row["value"])
   if row['attribute']=="hours":
      return 0.5*float(row['value'])

output_file_path = r'weighted_games.csv'
split_df = pd.DataFrame(columns=['app_id', 'user_id', 'review_id', 'attribute', 'value', 'date'])
split_df.to_csv(output_file_path, index=False, mode='w')

#interactions_df=pd.read_csv('split_games.csv')
for chunk1 in pd.read_csv('split_games.csv', chunksize=1000):
   #print(interactions_df)
   #interactions_df['value'] = interactions_df.apply(lambda x:evaluate_review(x),axis=1)
   rows1 = []
   for index1, row1 in chunk1.iterrows():
      rows1.append({'app_id': row1['app_id'], 'user_id': row1['user_id'], 'review_id': row1['review_id'], 'attribute': row1['attribute'], 'value': evaluate_review(row1), 'date': row1['date']})
    # 将当前块数据转换为DataFrame并写入文件
   chunk_split_df1 = pd.DataFrame(rows1)
   chunk_split_df1.to_csv(output_file_path, index=False, mode='a', header=False)
chunk_split_df1.head(100)

,app_id,user_id,review_id,attribute,value,date
0,412020,7179083,26268000.0,funny,0.00,2022-01-31
1,412020,7179083,26268000.0,helpful,0.00,2022-01-31
2,412020,7179083,26268000.0,is_recommended,1.00,2022-01-31
3,412020,7179083,26268000.0,hours,14.75,2022-01-31
4,476600,6792325,26268001.0,funny,0.00,2017-11-23
...,...,...,...,...,...,...
95,386180,2738482,26268023.0,hours,53.60,2021-11-29
96,346110,10394705,26268024.0,funny,0.00,2022-06-12
97,346110,10394705,26268024.0,helpful,0.00,2022-06-12
98,346110,10394705,26268024.0,is_recommended,1.00,2022-06-12


In [3]:
#chunk_split_df.size
interactions_df=pd.read_csv('weighted_games.csv')
#print(interactions_df.head(100))

In [4]:
interactions_df.head(100)

,app_id,user_id,review_id,attribute,value,date
0,975370,51580,0.0,funny,0.00,2022-12-12
1,975370,51580,0.0,helpful,0.00,2022-12-12
2,975370,51580,0.0,is_recommended,1.00,2022-12-12
3,975370,51580,0.0,hours,18.15,2022-12-12
4,304390,2586,1.0,funny,0.00,2017-02-17
...,...,...,...,...,...,...
95,518790,271318,23.0,hours,5.00,2022-12-31
96,236850,118252,24.0,funny,1.20,2021-03-20
97,236850,118252,24.0,helpful,6.90,2021-03-20
98,236850,118252,24.0,is_recommended,1.00,2021-03-20


In [5]:
users_interactions_count_df = interactions_df.groupby(['user_id', 'app_id']).size().groupby('user_id').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 6].reset_index()[['user_id']]
print('# users with at least 1 interactions: %d' % len(users_with_enough_interactions_df))

# users: 11176644
# users with at least 1 interactions: 838422


In [6]:
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'outer',
               left_on = 'user_id',
               right_on = 'user_id')

In [7]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['user_id', 'app_id'])['value'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(100)

# of unique user/item interactions: 26268047


,user_id,app_id,value
0,0,49520,4.197708
1,0,214490,4.658211
2,0,1454400,8.591335
3,1,897450,3.078951
4,2,291550,3.061776
...,...,...,...
95,76,880940,4.244126
96,76,960990,2.887525
97,76,1085660,7.589838
98,76,1172620,5.354029


In [12]:
values_cnt = interactions_full_df['user_id'].value_counts()
tbremove = values_cnt[values_cnt < 2].index
filtered_df = interactions_full_df[~interactions_full_df['user_id'].isin(tbremove)]

In [14]:
interactions_train_df, interactions_test_df = train_test_split(filtered_df,
                                   stratify=filtered_df['user_id'], 
                                   test_size=0.30,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 13713912
# interactions on Test set: 5877391


In [15]:
interactions_full_indexed_df = interactions_full_df.set_index('user_id')
interactions_train_indexed_df = interactions_train_df.set_index('user_id')
interactions_test_indexed_df = interactions_test_df.set_index('user_id')

In [16]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['contentId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [18]:
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(articles_df['contentId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['contentId'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['contentId'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()

In [20]:
#Computes the most popular items
item_popularity_df = interactions_full_df.groupby('app_id')['value'].sum().sort_values(ascending=False).reset_index()
item_popularity_df.head(10)

,app_id,value
0,440,2.015795e+06
1,252490,1.671931e+06
2,359550,1.287868e+06
3,730,1.282853e+06
4,1091500,1.178252e+06
5,374320,1.097600e+06
6,570,1.041525e+06
7,1085660,1.010551e+06
8,444090,1.007531e+06
9,227300,1.004562e+06


In [ ]:
# class PopularityRecommender:
    
#     MODEL_NAME = 'Popularity'
    
#     def __init__(self, popularity_df, items_df=None):
#         self.popularity_df = popularity_df
#         self.items_df = items_df
        
#     def get_model_name(self):
#         return self.MODEL_NAME
        
#     def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
#         # Recommend the more popular items that the user hasn't seen yet.
#         recommendations_df = self.popularity_df[~self.popularity_df['contentId'].isin(items_to_ignore)] \
#                                .sort_values('eventStrength', ascending = False) \
#                                .head(topn)

#         if verbose:
#             if self.items_df is None:
#                 raise Exception('"items_df" is required in verbose mode')

#             recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
#                                                           left_on = 'contentId', 
#                                                           right_on = 'contentId')[['eventStrength', 'contentId', 'title', 'url', 'lang']]


#         return recommendations_df
    
# popularity_model = PopularityRecommender(item_popularity_df, articles_df)

In [ ]:
# print('Evaluating Popularity recommendation model...')
# pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(popularity_model)
# print('\nGlobal metrics:\n%s' % pop_global_metrics)
# pop_detailed_results_df.head(10)

In [ ]:
# #Ignoring stopwords (words with no semantics) from English and Portuguese (as we have a corpus with mixed languages)
# stopwords_list = stopwords.words('english') + stopwords.words('portuguese')

# #Trains a model whose vectors size is 5000, composed by the main unigrams and bigrams found in the corpus, ignoring stopwords
# vectorizer = TfidfVectorizer(analyzer='word',
#                      ngram_range=(1, 2),
#                      min_df=0.003,
#                      max_df=0.5,
#                      max_features=5000,
#                      stop_words=stopwords_list)

# item_ids = articles_df['contentId'].tolist()
# tfidf_matrix = vectorizer.fit_transform(articles_df['title'] + "" + articles_df['text'])
# tfidf_feature_names = vectorizer.get_feature_names()
# tfidf_matrix

In [ ]:
# def get_item_profile(item_id):
#     idx = item_ids.index(item_id)
#     item_profile = tfidf_matrix[idx:idx+1]
#     return item_profile

# def get_item_profiles(ids):
#     item_profiles_list = [get_item_profile(x) for x in ids]
#     item_profiles = scipy.sparse.vstack(item_profiles_list)
#     return item_profiles

# def build_users_profile(person_id, interactions_indexed_df):
#     interactions_person_df = interactions_indexed_df.loc[person_id]
#     user_item_profiles = get_item_profiles(interactions_person_df['contentId'])
    
#     user_item_strengths = np.array(interactions_person_df['eventStrength']).reshape(-1,1)
#     #Weighted average of item profiles by the interactions strength
#     user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
#     user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
#     return user_profile_norm

# def build_users_profiles(): 
#     interactions_indexed_df = interactions_train_df[interactions_train_df['contentId'] \
#                                                    .isin(articles_df['contentId'])].set_index('personId')
#     user_profiles = {}
#     for person_id in interactions_indexed_df.index.unique():
#         user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
#     return user_profiles

In [ ]:
# user_profiles = build_users_profiles()
# len(user_profiles)

In [ ]:
# myprofile = user_profiles[-1479311724257856983]
# print(myprofile.shape)
# pd.DataFrame(sorted(zip(tfidf_feature_names, 
#                         user_profiles[-1479311724257856983].flatten().tolist()), key=lambda x: -x[1])[:20],
#              columns=['token', 'relevance'])

In [ ]:
# class ContentBasedRecommender:
    
#     MODEL_NAME = 'Content-Based'
    
#     def __init__(self, items_df=None):
#         self.item_ids = item_ids
#         self.items_df = items_df
        
#     def get_model_name(self):
#         return self.MODEL_NAME
        
#     def _get_similar_items_to_user_profile(self, person_id, topn=1000):
#         #Computes the cosine similarity between the user profile and all item profiles
#         cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
#         #Gets the top similar items
#         similar_indices = cosine_similarities.argsort().flatten()[-topn:]
#         #Sort the similar items by similarity
#         similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
#         return similar_items
        
#     def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
#         similar_items = self._get_similar_items_to_user_profile(user_id)
#         #Ignores items the user has already interacted
#         similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
#         recommendations_df = pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength']) \
#                                     .head(topn)

#         if verbose:
#             if self.items_df is None:
#                 raise Exception('"items_df" is required in verbose mode')

#             recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
#                                                           left_on = 'contentId', 
#                                                           right_on = 'contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]


#         return recommendations_df
    
# content_based_recommender_model = ContentBasedRecommender(articles_df)

In [ ]:
# print('Evaluating Content-Based Filtering model...')
# cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
# print('\nGlobal metrics:\n%s' % cb_global_metrics)
# cb_detailed_results_df.head(10)

In [22]:
#Creating a sparse pivot table with users in rows and items in columns
users_items_pivot_matrix_df = interactions_train_df.pivot(index='user_id', 
                                                          columns='app_id', 
                                                          values='value').fillna(0)

users_items_pivot_matrix_df.head(10)

/tmp/ipykernel_91834/2099652372.py:2: PerformanceWarning: The following operation may generate 13972189500 cells in the resulting pandas object.
  users_items_pivot_matrix_df = interactions_train_df.pivot(index='user_id',


: 

In [ ]:
users_items_pivot_matrix = users_items_pivot_matrix_df.as_matrix()
users_items_pivot_matrix[:10]

In [ ]:
users_ids = list(users_items_pivot_matrix_df.index)
users_ids[:10]

In [ ]:
users_items_pivot_sparse_matrix = csr_matrix(users_items_pivot_matrix)
users_items_pivot_sparse_matrix

In [ ]:
#The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15
#Performs matrix factorization of the original user item matrix
#U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)
U, sigma, Vt = svds(users_items_pivot_sparse_matrix, k = NUMBER_OF_FACTORS_MF)

In [ ]:
U.shape
Vt.shape
sigma = np.diag(sigma)
sigma.shape

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

In [ ]:
all_user_predicted_ratings_norm = (all_user_predicted_ratings - all_user_predicted_ratings.min()) / (all_user_predicted_ratings.max() - all_user_predicted_ratings.min())
#Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings_norm, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()
cf_preds_df.head(10)
len(cf_preds_df.columns)

In [21]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['contentId'].isin(items_to_ignore)] \
                               .sort_values('recStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df
    
cf_recommender_model = CFRecommender(cf_preds_df, articles_df)

NameError: name 'cf_preds_df' is not defined

In [ ]:
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)
print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

In [ ]:
class HybridRecommender:
    
    MODEL_NAME = 'Hybrid'
    
    def __init__(self, cb_rec_model, cf_rec_model, items_df, cb_ensemble_weight=1.0, cf_ensemble_weight=1.0):
        self.cb_rec_model = cb_rec_model
        self.cf_rec_model = cf_rec_model
        self.cb_ensemble_weight = cb_ensemble_weight
        self.cf_ensemble_weight = cf_ensemble_weight
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        #Getting the top-1000 Content-based filtering recommendations
        cb_recs_df = self.cb_rec_model.recommend_items(user_id, items_to_ignore=items_to_ignore, verbose=verbose,
                                                           topn=1000).rename(columns={'recStrength': 'recStrengthCB'})
        
        #Getting the top-1000 Collaborative filtering recommendations
        cf_recs_df = self.cf_rec_model.recommend_items(user_id, items_to_ignore=items_to_ignore, verbose=verbose, 
                                                           topn=1000).rename(columns={'recStrength': 'recStrengthCF'})
        
        #Combining the results by contentId
        recs_df = cb_recs_df.merge(cf_recs_df,
                                   how = 'outer', 
                                   left_on = 'contentId', 
                                   right_on = 'contentId').fillna(0.0)
        
        #Computing a hybrid recommendation score based on CF and CB scores
        #recs_df['recStrengthHybrid'] = recs_df['recStrengthCB'] * recs_df['recStrengthCF'] 
        recs_df['recStrengthHybrid'] = (recs_df['recStrengthCB'] * self.cb_ensemble_weight) \
                                     + (recs_df['recStrengthCF'] * self.cf_ensemble_weight)
        
        #Sorting recommendations by hybrid score
        recommendations_df = recs_df.sort_values('recStrengthHybrid', ascending=False).head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrengthHybrid', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df
    
hybrid_recommender_model = HybridRecommender(content_based_recommender_model, cf_recommender_model, articles_df,
                                             cb_ensemble_weight=1.0, cf_ensemble_weight=100.0)

In [ ]:
print('Evaluating Hybrid model...')
hybrid_global_metrics, hybrid_detailed_results_df = model_evaluator.evaluate_model(hybrid_recommender_model)
print('\nGlobal metrics:\n%s' % hybrid_global_metrics)
hybrid_detailed_results_df.head(10)

In [ ]:
global_metrics_df = pd.DataFrame([cb_global_metrics, pop_global_metrics, cf_global_metrics, hybrid_global_metrics]) \
                        .set_index('modelName')
global_metrics_df

In [ ]:
%matplotlib inline
ax = global_metrics_df.transpose().plot(kind='bar', figsize=(15,8))
for p in ax.patches:
    ax.annotate("%.3f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')

In [ ]:
def inspect_interactions(person_id, test_set=True):
    if test_set:
        interactions_df = interactions_test_indexed_df
    else:
        interactions_df = interactions_train_indexed_df
    return interactions_df.loc[person_id].merge(articles_df, how = 'left', 
                                                      left_on = 'contentId', 
                                                      right_on = 'contentId') \
                          .sort_values('eventStrength', ascending = False)[['eventStrength', 
                                                                          'contentId',
                                                                          'title', 'url', 'lang']]

In [ ]:
inspect_interactions(-1479311724257856983, test_set=False).head(20)

In [ ]:
hybrid_recommender_model.recommend_items(-1479311724257856983, topn=20, verbose=True)